# Product Recommendation
Reference: https://ieeexplore.ieee.org/document/5430993

In [49]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

In [50]:
tf.__version__

'2.2.0'

## Data Preprocessing

In [51]:
Y_data = pd.read_csv('data/Y.csv', header=None, names=['Rating','Movie','User'], dtype=int) # training data
P_data = pd.read_csv('data/P.csv', header=None, names=['Rating','Movie','User'], dtype=int) # test data ('probe-set' mentioned in paper)

In [52]:
display(Y_data.head())
display(P_data.head())

,Rating,Movie,User
0,5,2,1
1,4,7,1
2,4,8,1
3,4,11,1
4,4,12,1


,Rating,Movie,User
0,3,6,1
1,5,96,1
2,3,1,2
3,3,33,3
4,5,42,4


In [53]:
Y_data.shape, P_data.shape

((3399874, 3), (189699, 3))

In [54]:
print(Y_data['Rating'].unique().max(), Y_data['Movie'].unique().max(), Y_data['User'].unique().max())
print(P_data['Rating'].unique().max(), P_data['Movie'].unique().max(), P_data['User'].unique().max())

5 100 137328
5 100 137328


In [55]:
k, n = Y_data['Movie'].unique().max(), Y_data['User'].unique().max()
k, n

(100, 137328)

In [56]:
indices = np.reshape(Y_data[['Movie', 'User']].values-1, (-1, 2))
print(indices.shape)
indices

(3399874, 2)


array([[     1,      0],
       [     6,      0],
       [     7,      0],
       ...,
       [    97, 137327],
       [    98, 137327],
       [    99, 137327]])

In [57]:
Z_sparse = tf.SparseTensor(indices=indices, values=Y_data['Rating'].values, dense_shape=[k, n])
Z_sparse = tf.cast(Z_sparse, tf.float64)

In [10]:
# %%time
# t = 0
# Z_t = tf.sparse.slice(Z_sparse, [0, t], [100, 1])

CPU times: user 19.2 ms, sys: 1.05 ms, total: 20.3 ms
Wall time: 18.8 ms


In [58]:
# use dense matrices for faster linear transformations since all matrices can fit in memory
Z = tf.sparse.to_dense(Z_sparse, validate_indices=False)
Z

<tf.Tensor: shape=(100, 137328), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 3.],
       [0., 0., 0., ..., 3., 0., 0.],
       ...,
       [5., 0., 0., ..., 4., 0., 4.],
       [4., 0., 3., ..., 0., 0., 4.],
       [3., 4., 0., ..., 4., 5., 4.]])>

In [60]:
# memoization
t_Z_dict = {}
t_y_dict = {}
t_x_dict = {}
t_Hy_dict = {}
t_Hx_dict = {}
t_Hy_trans_dict = {}
t_Hx_trans_dict = {}
t_movie_ids_labels_dict = {}
t_labels_dict = {}

for t in tqdm(range(n)):
    movie_ids = Y_data['Movie'][Y_data['User']==t+1].values
    H_yt = tf.constant(np.identity(k)[movie_ids-1], dtype=tf.float64)
    H_xt = tf.constant(np.delete(np.identity(k), movie_ids-1, 0), dtype=tf.float64)
    Z_t = tf.expand_dims(Z[:, t], axis=1) # alternative: Z_t = tf.sparse.slice(Z_sparse, [0, t], [100, 1]) 
    y_t = tf.matmul(H_yt, Z_t)
    x_t = tf.matmul(H_xt, Z_t)
    
    # store the variables for fast future reference
    t_Hy_dict[t] = H_yt
    t_Hx_dict[t] = H_xt
    t_Hx_trans_dict[t] = tf.transpose(H_xt)
    t_Hy_trans_dict[t] = tf.transpose(H_yt)
    
    t_x_dict[t] = x_t
    t_y_dict[t] = y_t
    t_Z_dict[t] = Z_t
    
    t_movie_ids_labels_dict[t] = P_data['Movie'][P_data['User']==t+1].values
    t_labels_dict[t] = tf.expand_dims(P_data['Rating'][P_data['User']==t+1].values, axis=1)

100%|██████████| 137328/137328 [13:45<00:00, 166.34it/s]


## Initialization
$\mu$ has 1 type available <br />
R has 4 types available

In [61]:
# initial estimate of mu
N = 0
H_yty_t = 0

for t in tqdm(range(n)):
    N += tf.matmul(t_Hy_trans_dict[t], t_Hy_dict[t])
    H_yty_t += tf.matmul(t_Hy_trans_dict[t], t_y_dict[t])

100%|██████████| 137328/137328 [00:15<00:00, 8605.84it/s] 


In [62]:
# The ith diagonal element of N equals the total number of ratings of the ith product.
N

<tf.Tensor: shape=(100, 100), dtype=float64, numpy=
array([[20017.,     0.,     0., ...,     0.,     0.,     0.],
       [    0., 23917.,     0., ...,     0.,     0.,     0.],
       [    0.,     0., 31634., ...,     0.,     0.,     0.],
       ...,
       [    0.,     0.,     0., ..., 60896.,     0.,     0.],
       [    0.,     0.,     0., ...,     0., 61521.,     0.],
       [    0.,     0.,     0., ...,     0.,     0., 64506.]])>

In [63]:
print(H_yty_t.shape)
mu_hat0 = tf.matmul(tf.linalg.inv(N), H_yty_t)
mu_hat0.shape

(100, 1)


TensorShape([100, 1])

In [64]:
# initial estimates of R (4 types available)
R_hat0_1 = tf.constant(np.identity(k), dtype=tf.float64)
R_hat0_1

<tf.Tensor: shape=(100, 100), dtype=float64, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])>

In [69]:
S = 0
for t in tqdm(range(n)):
    Hyt = t_Hy_dict[t]
    yt = t_y_dict[t]
    Hytmu_hat0 = tf.matmul(Hyt, mu_hat0)
    S += tf.matmul(tf.transpose(Hyt), tf.matmul(yt - Hytmu_hat0, tf.matmul(tf.transpose(yt - Hytmu_hat0), Hyt)))

100%|██████████| 137328/137328 [00:31<00:00, 4384.91it/s]


In [70]:
# diag_S is the diagonal matrix consisting of the diagonal elements of S
diag_S = tf.linalg.diag(tf.linalg.tensor_diag_part(S))
R_hat0_2 = tf.matmul(tf.linalg.inv(N), diag_S)
R_hat0_2

<tf.Tensor: shape=(100, 100), dtype=float64, numpy=
array([[1.72440427, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.94219113, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.43659411, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.18291506, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.03485685,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.26227449]])>

In [71]:
# R_hat0_3 is not a good initializer when rating variances are far from one
R_hat0_3 = tf.matmul(tf.linalg.sqrtm(tf.linalg.inv(diag_S)), tf.matmul(S, tf.linalg.sqrtm(tf.linalg.inv(diag_S))))
R_hat0_3

<tf.Tensor: shape=(100, 100), dtype=float64, numpy=
array([[ 1.        ,  0.07418256, -0.01158277, ..., -0.01462987,
        -0.02215371, -0.01844816],
       [ 0.07418256,  1.        ,  0.03674347, ...,  0.0256191 ,
         0.03563234,  0.03926307],
       [-0.01158277,  0.03674347,  1.        , ...,  0.10955311,
         0.12823359,  0.15560634],
       ...,
       [-0.01462987,  0.0256191 ,  0.10955311, ...,  1.        ,
         0.19781317,  0.15164928],
       [-0.02215371,  0.03563234,  0.12823359, ...,  0.19781317,
         1.        ,  0.18995689],
       [-0.01844816,  0.03926307,  0.15560634, ...,  0.15164928,
         0.18995689,  1.        ]])>

In [72]:
R_hat0_4 = tf.matmul(tf.linalg.sqrtm(tf.linalg.inv(N)), tf.matmul(S, tf.linalg.sqrtm(tf.linalg.inv(N))))
R_hat0_4

<tf.Tensor: shape=(100, 100), dtype=float64, numpy=
array([[ 1.72440427,  0.09455639, -0.01823052, ..., -0.02089473,
        -0.02959417, -0.02721758],
       [ 0.09455639,  0.94219113,  0.04274809, ...,  0.02704644,
         0.03518471,  0.04281842],
       [-0.01823052,  0.04274809,  1.43659411, ...,  0.14281326,
         0.15635399,  0.20954206],
       ...,
       [-0.02089473,  0.02704644,  0.14281326, ...,  1.18291506,
         0.21886288,  0.18530794],
       [-0.02959417,  0.03518471,  0.15635399, ...,  0.21886288,
         1.03485685,  0.21710614],
       [-0.02721758,  0.04281842,  0.20954206, ...,  0.18530794,
         0.21710614,  1.26227449]])>

## Expectation Maximization Algorithm

In [113]:
LOG_2PI = tf.math.log(2*tf.constant(np.pi, dtype=tf.float64))

@tf.function(experimental_relax_shapes=True)
def run_graph_em(mu, R, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans):
    # for R estimation
    R_xt = H_xt @ R @ H_xt_trans
    R_yt = H_yt @ R @ H_yt_trans
    R_yt_det = tf.linalg.det(R_yt)
    R_yt_inv = tf.linalg.inv(R_yt)
    R_xtyt = H_xt @ R @ H_yt_trans

    mu_yt = tf.matmul(H_yt, mu)
    mu_xt = tf.matmul(H_xt, mu)

    X_t_hat = R_xtyt @ R_yt_inv @ (y_t - mu_yt) + mu_xt
    Z_t_hat = H_yt_trans @ y_t + H_xt_trans @ X_t_hat
    
    R_hat_sum_part = (Z_t_hat - mu) @ tf.transpose(Z_t_hat - mu) \
                        + H_xt_trans @ (R_xt - R_xtyt @ R_yt_inv @ tf.transpose(R_xtyt)) @ H_xt

    # for mu estimation
    Hyt_trans_Ryt_inv_Hyt_sum_part = H_yt_trans @ R_yt_inv @ H_yt
    Hyt_trans_Ryt_inv_yt_sum_part = H_yt_trans @ R_yt_inv @ y_t

    # for log likelihood calculation
    log_p_hat_part = -1/2*(tf.math.log(R_yt_det) + tf.transpose(y_t - mu_yt) @ R_yt_inv @ (y_t - mu_yt) + k*LOG_2PI) 
    
    return R_hat_sum_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part

In [104]:
def expectation_maximization(mu, R):
    Hyt_trans_Ryt_inv_Hyt_sum = 0
    Hyt_trans_Ryt_inv_yt_sum = 0
    R_hat_sum = 0
    log_p_hat = 0
    
    for t in tqdm(range(n)):
        y_t = t_y_dict[t]
        H_xt = t_Hx_dict[t]
        H_xt_trans = t_Hx_trans_dict[t]
        H_yt = t_Hy_dict[t]
        H_yt_trans = t_Hy_trans_dict[t]
        
        R_hat_sum_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part = \
            run_graph_em(mu, R, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans)
        
        R_hat_sum += R_hat_sum_part
        Hyt_trans_Ryt_inv_Hyt_sum += Hyt_trans_Ryt_inv_Hyt_sum_part
        Hyt_trans_Ryt_inv_yt_sum += Hyt_trans_Ryt_inv_yt_sum_part
        log_p_hat += log_p_hat_part
        
    R_hat = R_hat_sum / n
    mu_hat = tf.matmul(tf.linalg.inv(Hyt_trans_Ryt_inv_Hyt_sum), Hyt_trans_Ryt_inv_yt_sum)    
    return mu_hat, R_hat, log_p_hat

In [106]:
delta = 0.0005
mu = mu_hat0
R = R_hat0_4
log_p = tf.constant(-np.inf, dtype=tf.float64)

for i in range(30):
    if i % 5 == 0:
        print(f'iteration: {i}')
    
    mu_hat, R_hat, log_p_hat = expectation_maximization(mu, R)
    convergence_criterion = log_p_hat/n - log_p/n < delta
    
    print('normalized log_p_hat:', (log_p_hat/n).numpy())
    print('normalized log_p:', (log_p/n).numpy())
    print('convergence gap:', (log_p_hat/n - log_p/n).numpy())
    
    if convergence_criterion:
        break
        
    # use new estimattions for next iteration
    mu = mu_hat
    R = R_hat
    log_p = log_p_hat

  0%|          | 254/137328 [00:00<01:52, 1216.96it/s]

iteration: 0


  0%|          | 133/137328 [00:00<01:43, 1322.67it/s]

normalized log_p_hat: [[-101.37606558]]
normalized log_p: -inf
convergence gap: [[inf]]


  0%|          | 132/137328 [00:00<01:44, 1311.52it/s]

...

normalized log_p_hat: [[-100.55847312]]
normalized log_p: [[-100.55918673]]
convergence gap: [[0.00071361]]


  0%|          | 110/137328 [00:00<02:05, 1097.32it/s]

normalized log_p_hat: [[-100.55788391]]
normalized log_p: [[-100.55847312]]
convergence gap: [[0.00058921]]
iteration: 25


100%|██████████| 137328/137328 [01:30<00:00, 1514.13it/s]

normalized log_p_hat: [[-100.5573955]]
normalized log_p: [[-100.55788391]]
convergence gap: [[0.00048841]]


In [107]:
# 26 iterations, ~39 min
np.save('results/em_mu.npy', mu_hat)
np.save('results/em_R.npy', R_hat)
np.save('results/em_log_p.npy', log_p_hat)

## McMichael’s Algorithm

In [114]:
@tf.function(experimental_relax_shapes=True)
def run_graph_mcmichael(mu, R, y_t, H_yt, H_yt_trans):
    # for R estimation
    R_yt = H_yt @ R @ H_yt_trans
    R_yt_det = tf.linalg.det(R_yt)
    R_yt_inv = tf.linalg.inv(R_yt)
    mu_yt = tf.matmul(H_yt, mu)
    log_p_gradient_part = H_yt_trans @ (R_yt_inv - R_yt_inv @ (y_t - mu_yt) @ tf.transpose(y_t - mu_yt) @ R_yt_inv) @ H_yt

    # for mu estimation
    Hyt_trans_Ryt_inv_Hyt_sum_part = H_yt_trans @ R_yt_inv @ H_yt
    Hyt_trans_Ryt_inv_yt_sum_part = H_yt_trans @ R_yt_inv @ y_t
    
    # for log likelihood calculation
    log_p_hat_part = -1/2*(tf.math.log(R_yt_det) + tf.transpose(y_t - mu_yt) @ R_yt_inv @ (y_t - mu_yt) + k*LOG_2PI)
    
    return log_p_gradient_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part

In [115]:
def mcmichael(mu, R):
    gamma = 0.00001
    Hyt_trans_Ryt_inv_Hyt_sum = 0
    Hyt_trans_Ryt_inv_yt_sum = 0
    log_p_gradient = 0
    log_p_hat = 0

    for t in tqdm(range(n)):
        y_t = t_y_dict[t]
        H_yt = t_Hy_dict[t]
        H_yt_trans = t_Hy_trans_dict[t]
        
        log_p_gradient_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part = \
            run_graph_mcmichael(mu, R, y_t, H_yt, H_yt_trans)
        
        log_p_gradient += log_p_gradient_part
        Hyt_trans_Ryt_inv_Hyt_sum += Hyt_trans_Ryt_inv_Hyt_sum_part
        Hyt_trans_Ryt_inv_yt_sum += Hyt_trans_Ryt_inv_yt_sum_part
        log_p_hat += log_p_hat_part
        
    R_hat = R + gamma*(R @ (-1/2*log_p_gradient) @ R)
    mu_hat = tf.matmul(tf.linalg.inv(Hyt_trans_Ryt_inv_Hyt_sum), Hyt_trans_Ryt_inv_yt_sum)
    return mu_hat, R_hat, log_p_hat

In [119]:
delta = 0.0005
mu = mu_hat0
R = R_hat0_4
log_p = tf.constant(-np.inf, dtype=tf.float64)

for i in range(40):
    if i % 5 == 0:
        print(f'iteration: {i}')
    
    mu_hat, R_hat, log_p_hat = mcmichael(mu, R)
    convergence_criterion = log_p_hat/n - log_p/n < delta
    
    print('normalized log_p_hat:', (log_p_hat/n).numpy())
    print('normalized log_p:', (log_p/n).numpy())
    print('convergence gap:', (log_p_hat/n - log_p/n).numpy())
    
    if convergence_criterion:
        break
        
    # use new estimattions for next iteration
    mu = mu_hat
    R = R_hat
    log_p = log_p_hat

  0%|          | 165/137328 [00:00<01:23, 1649.57it/s]

iteration: 0


  0%|          | 193/137328 [00:00<01:11, 1927.05it/s]

normalized log_p_hat: [[-101.37606558]]
normalized log_p: -inf
convergence gap: [[inf]]


  0%|          | 196/137328 [00:00<01:10, 1947.64it/s]

...

normalized log_p_hat: [[-100.55950462]]
normalized log_p: [[-100.56012163]]
convergence gap: [[0.00061701]]


  0%|          | 214/137328 [00:00<01:04, 2134.16it/s]

normalized log_p_hat: [[-100.5589649]]
normalized log_p: [[-100.55950462]]
convergence gap: [[0.00053972]]


100%|██████████| 137328/137328 [01:01<00:00, 2224.25it/s]

normalized log_p_hat: [[-100.55849192]]
normalized log_p: [[-100.5589649]]
convergence gap: [[0.00047298]]


In [121]:
# 35 iterations, ~35 min
np.save('results/mcmichael_mu.npy', mu_hat)
np.save('results/mcmichael_R.npy', R_hat)
np.save('results/mcmichael_log_p.npy', log_p_hat)

## Evaluation

In [108]:
def evaluate(mu, R):
    square_error = 0
    l = 0
    for t in tqdm(range(n)):
        movie_ids_t = t_movie_ids_labels_dict[t]
        labels_t = t_labels_dict[t]

        # calculate X_t_hat
        y_t = t_y_dict[t]
        H_xt = t_Hx_dict[t]
        H_xt_trans = t_Hx_trans_dict[t]
        H_yt = t_Hy_dict[t]
        H_yt_trans = t_Hy_trans_dict[t]
        
        R_xt = H_xt @ R @ H_xt_trans
        R_yt = H_yt @ R @ H_yt_trans
        R_yt_inv = tf.linalg.inv(R_yt)
        R_xtyt = H_xt @ R @ H_yt_trans
        
        mu_yt = tf.matmul(H_yt, mu)
        mu_xt = tf.matmul(H_xt, mu)

        X_t_hat = R_xtyt @ R_yt_inv @ (y_t - mu_yt) + mu_xt
        
        # clip ratings
        predictions_t = tf.matmul(H_xt_trans, X_t_hat).numpy()[movie_ids_t-1]
        predictions_t = np.clip(predictions_t, 1, 5)
        
        # accumulate square_error and l
        square_error += tf.matmul(tf.transpose(labels_t - predictions_t), labels_t - predictions_t)
        l += len(labels_t)
    return np.sqrt(square_error/l)

In [109]:
em_mu = np.load('results/em_mu.npy')
em_R = np.load('results/em_R.npy')
rmse = evaluate(em_mu, em_R)
rmse

100%|██████████| 137328/137328 [02:02<00:00, 1125.11it/s]


array([[1.07507623]])

In [122]:
mcmichael_mu = np.load('results/mcmichael_mu.npy')
mcmichael_R = np.load('results/mcmichael_R.npy')
rmse = evaluate(mcmichael_mu, mcmichael_R)
rmse

100%|██████████| 137328/137328 [01:56<00:00, 1178.34it/s]


array([[1.07510565]])